In [ ]:
#in case of not having this package
#! conda install pillow

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import cv2

In [2]:
data_folder = "att_faces/s"

#### first of all we use the first 5 pictures of each class for feature extraction using PCA .
#### In other words we use the first 5 pictures for dimentionality reduction using PCA .
#### Note that the dataset has 40 classes with 10 pictures so it has 400 images or instance.
#### pictures' dimensions are 1024 by 1024 so we resize them to 32 by 32 to get 1024 pixels or features

In [3]:
for epoch in range(20):
    # train
    target = []
    X = np.array([],dtype=int)
    for num in range(1,41):
        #trainList = np.random.choice(range(1,11), 5, replace=False)
        trainList = np.array([1,2,3,4,5])
        for j in trainList:
            file_path = join(data_folder + str(num), str(j) + ".pgm")
            #file_path = "att_faces/s%d/%d.pgm" % (num,j)
            loaded_image=np.array(plt.imread(file_path))
            image_32x32 = cv2.resize(loaded_image, (32, 32), interpolation=cv2.IMREAD_GRAYSCALE)
            image_1024 = image_32x32.reshape(-1)
            X = np.concatenate((X,image_1024),axis=0)
            target.append(num)
    X = np.reshape(X,(200,1024))
    #X = X.T #? :/

n ,p = np.shape(X)    
print("sampels: %d , attrs: %d" % (n,p))
target = np.array(target)
y = target
print(y.shape)
        

sampels: 200 , attrs: 1024
(200,)


In [4]:
from sklearn.preprocessing import StandardScaler

standardized_data = StandardScaler().fit_transform(X)
#print(scaler.fit(X))
print(standardized_data.shape)
print("variance of features: ",np.var(standardized_data,axis=0))
sample_data = standardized_data
#print(scaler.transform(X)) # Here we should transform test data via scaler
R_x = np.matmul(sample_data.T, sample_data) # It should be checked
print('Covariance Matrix :\n',R_x)
print('\n\n')
print('shape of Covariance Matrix :\n',R_x.shape)



(200, 1024)
variance of features:  [1. 1. 1. ... 1. 1. 1.]
Covariance Matrix :
 [[200.         197.81472344 197.11249131 ... -20.80814811  10.68487359
  -10.43091112]
 [197.81472344 200.         198.09024534 ... -25.44000651   7.11263237
  -12.08494327]
 [197.11249131 198.09024534 200.         ... -27.89989016   7.02732701
  -10.6697906 ]
 ...
 [-20.80814811 -25.44000651 -27.89989016 ... 200.         151.30418051
  143.02810981]
 [ 10.68487359   7.11263237   7.02732701 ... 151.30418051 200.
  168.16208956]
 [-10.43091112 -12.08494327 -10.6697906  ... 143.02810981 168.16208956
  200.        ]]



shape of Covariance Matrix :
 (1024, 1024)


In [5]:
from scipy.linalg import eigh
import sys
# np.set_printoptions(threshold=sys.maxsize)
eigen_values, eigen_vectors = eigh(R_x, eigvals=(1019,1023))

print('Shape of Eigen Vectors : \n',eigen_vectors.shape)
print('Max 5 Eigen Values : \n',eigen_values)

A = eigen_vectors
# print('5 Eigen Vectors from 5 max eigen values : \n',eigen_vectors)
print('Updated shape of Eigen Vectors : \n',A.shape)



Shape of Eigen Vectors : 
 (1024, 5)
Max 5 Eigen Values : 
 [10352.89612334 11642.16106012 18281.96562144 27513.72134189
 32713.03590476]
Updated shape of Eigen Vectors : 
 (1024, 5)


In [15]:
new_coordinates = np.matmul(sample_data,A)

print()
print( 'shape of eigen vectors ---' , ' shape of traversed sample_data ---' , ' shape of new coordinates')
print()
print( '\t',A.shape,'\t,\t   ', sample_data.T.shape,'\t,\t', new_coordinates.shape)
print('\n',' '*20,'*'*45,'\n')
print('\t\t\t',sample_data.shape,' x ',A.shape ,' = ', new_coordinates.shape)
print()
print(new_coordinates.shape)


shape of eigen vectors ---  shape of traversed sample_data ---  shape of new coordinates

	 (1024, 5) 	,	    (1024, 200) 	,	 (200, 5)

                      ********************************************* 

			 (200, 1024)  x  (1024, 5)  =  (200, 5)

(200, 5)


In [16]:
new_coordinates = np.hstack((new_coordinates, y))

In [17]:
columns=("1st_principal","2nd_principal","3rd_principal","4th_principal","5th_principal","label")
dataframe = pd.DataFrame(data=new_coordinates, columns=columns)
print("*"*80)
print(dataframe.head(10))

********************************************************************************
   1st_principal  2nd_principal  3rd_principal  4th_principal  5th_principal  \
0       3.666213       4.484835      18.610272      -0.613120      14.128133   
1      20.767415       0.038980      -1.646347       5.595974      24.820460   
2       4.992645       6.813258      10.171215       7.769072      19.192764   
3       3.412162     -21.033907       4.221059       7.767411      24.919410   
4       4.349715     -13.928596       5.234321      11.823780      26.338073   
5      -0.665908      -1.911835      13.325572       9.223935      -0.200505   
6       1.872696       5.065024      14.309759       7.209565      -0.023390   
7      -1.144515       2.307209      11.491299       9.740701       0.065005   
8       2.460077       4.493274      13.707785       8.731214      -1.176714   
9       5.485019       1.925007      13.252942       7.552715       0.926405   

   label  
0    1.0  
1    1.0  
2    

till now, we implement PCA on the first 5 images of each class.
let's do it again and use 5 random images from each class instead to get better result.
then we use classifier to recognize each picture's class .
our classifier will be KNN.